In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
!pip install transformers>=4.42.3


In [ ]:
from transformers import pipeline

pipe = pipeline("translation", model="google-t5/t5-3b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/11.4G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1140: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
model_name = "google/t5-v1_1-3b"


tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-3b")
model = AutoModelWithLMHead.from_pretrained("google-t5/t5-3b").to("cuda")
pipe =  pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    framework="pt",  # PyTorch backend
    device=0
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1833: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausa

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google-t5/t5-3b"
tokenizer = T5Tokenizer.from_pretrained(model_name,legacy=False)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [ ]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/100_enron_spam_data.csv')
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
195,195,22338,not for sale in the u . a . e - usb voip hands...,not for sale in the u . a . e\nusb voip handse...,spam,2005-07-27
196,196,6329,re : improve communication,"zimin ,\ni will make sure that you have the op...",ham,2000-06-22
197,197,33146,giga medzz,dear sir / madam .\nwe are pleased to introduc...,spam,2005-06-11
198,198,24384,improved health aegjngkjgw,go\nbig . . . . . . supersize today ! ! !\n* *...,spam,2002-07-05


In [ ]:
df = df[:120]

In [ ]:
generation_args = {
    "max_new_tokens": 350,
    "temperature": 0.3,
    "do_sample": True,
}

In [ ]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = """I want you to act as an email classifier.Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. Text: {} Answer with: Spam or Ham only.""".format(row["Message"])

    inputs = tokenizer(text_template, return_tensors="pt", max_length=512, truncation=True)


    #outputs = model.generate(**inputs, max_length=10)
    # Call your pipe function (replace `pipe` and `generation_args` with actual implementation)
    #output = pipe(text_template, **generation_args)

    # Extract the generated text
    #generated_text = output[0]["generated_text"]

    outputs = model.generate(**inputs, max_length=10)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


example 0 : 

'kirstee hewitt'


'?'


'ham'
example 1 : 

'Text: online credit breakthroughrepair your'


'?'


'spam'
example 2 : 

'::.'


'?'


'ham'
example 3 : 

'Text: please incease the bonus for'


'?'


'ham'
example 4 : 

"Text: working on your company '"


'?'


'spam'
example 5 : 

"i ' m a"


'?'


'spam'
example 6 : 

'li xiao referred ale'


'?'


'ham'
example 7 : 

'Text: soft at incredibly low prices the'


'?'


'spam'
example 8 : 

'Text: louise , attached is'


'?'


'ham'
example 9 : 

'Text: welcome to offshorepharm ,'


'?'


'spam'


KeyboardInterrupt: 

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-cpu


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.0/230.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: ke

In [ ]:
from huggingface_hub import login
import os

# Get the Hugging Face token from Colab secrets
hf_token = os.getenv('Pajama')

# Login to Hugging Face
login(token=hf_token)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-3B-v1")

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    framework="pt",
)

In [ ]:
generation_args = {
    "max_new_tokens": 350,
    "temperature": 0.3,
    "do_sample": True,
}

In [ ]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = """Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. Text: {} Class of the email:?""".format(row["Message"])

    # Set up the messages for the assistant
    messages = [
        {"role": "system", "content": "You are a clever assistant which helps answering general questions and also math and logic questions."},
        {"role": "user", "content": text_template},
    ]

    # Call your pipe function (replace `pipe` and `generation_args` with actual implementation)
    output = pipe(messages, **generation_args)

    # Extract the generated text
    generated_text = output[0]["generated_text"]
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


example 0 : 

[{'content': 'You are a clever assistant which helps answering general '
             'questions and also math and logic questions.',
  'role': 'system'},
 {'content': 'Classify the text into Spam or Ham. Please provide me at the end '
             'of the response with Class of the email:. Text: vince :\n'
             'this is precisely the concern i have for kirstee . steve tells '
             'me that she is\n'
             'very imaginative and eager to try to add value , but she has no '
             'local\n'
             'supervisor to help push some of her ideas . i will give you an '
             'update of our\n'
             'discussion later and solicit your suggestions .\n'
             'grant .\n'
             '- - - - - - - - - - - - - - - - - - - - - - forwarded by grant '
             'masson / hou / ect on 07 / 25 / 2000 09 : 07\n'
             'am - - - - - - - - - - - - - - - - - - - - - - - - - - -\n'
             'kirstee hewitt\n'
             '07

KeyboardInterrupt: 

In [ ]:
from huggingface_hub import login
import os

# Get the Hugging Face token from Colab secrets
hf_token = os.getenv('Pajama')

# Login to Hugging Face
login(token=hf_token)

In [ ]:
import torch

In [ ]:
#from transformers import pipeline

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
model_name = "h2oai/h2o-danube3-4b-chat"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)


tokenizer_config.json:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/484 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:

model_name = "h2oai/h2o-danube3-4b-chat"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

messages = [
    {"role": "user", "content": "Why is drinking water so healthy?"},
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tokenizer(
    prompt, return_tensors="pt", add_special_tokens=False
).to("cuda")

# generate configuration can be modified to your needs
tokens = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    min_new_tokens=2,
    max_new_tokens=256,
)[0]

tokens = tokens[inputs["input_ids"].shape[1]:]
answer = tokenizer.decode(tokens, skip_special_tokens=True)
print(answer)


In [ ]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []
email_class = ""
# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    text_template = """Classify the text into Spam or Ham. Please provide me at the end of the response with Class of the email:. Text: {} Class of the email:?""".format(row["Message"])

    # Set up the messages for the assistant
    messages = [
        {"role": "user", "content": text_template},
    ]


    prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    )
    inputs = tokenizer(
    prompt, return_tensors="pt", add_special_tokens=False
    ).to("cuda")

    # generate configuration can be modified to your needs
    tokens = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          min_new_tokens=2,
          max_new_tokens=256,
          )[0]

    tokens = tokens[inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(tokens, skip_special_tokens=True)
    #print(answer)


    # Extract the generated text
    generated_text = answer
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


example 0 : 

'Spam'


''


'ham'
example 1 : 

'Class of the email: Spam'


'Spam'


'spam'
example 2 : 

'Spam'


'spam'


'ham'
example 3 : 

'Spam'


'spam'


'ham'
example 4 : 

'Class of the email: Spam'


'Spam'


'spam'
example 5 : 

'Class of the email: Spam'


'Spam'


'spam'
example 6 : 

'Spam'


'spam'


'ham'
example 7 : 

'Class of the email: Spam'


'Spam'


'spam'
example 8 : 

'Spam'


'spam'


'ham'
example 9 : 

'Class of the email: Spam'


'Spam'


'spam'
example 10 : 

'Spam'


'spam'


'spam'
example 11 : 

'Spam'


'spam'


'ham'
example 12 : 

'Spam'


'spam'


'spam'
example 13 : 

'Spam'


'spam'


'ham'
example 14 : 

'Spam'


'spam'


'ham'
example 15 : 

('The text provided appears to be a mix of random words and phrases, making it '
 'difficult to classify as either spam or ham. It seems to contain elements of '
 'both, with some words and phrases that could be considered spammy (e.g., '
 '"pornography," "saucepan"), but also some that could be considered

In [ ]:
true_labels = true_labels[1:120]
predicted_labels = predicted_labels[1:120]

In [ ]:
label_mapping = {'ham': 0, 'spam': 1}
true_labels = [label_mapping[label] for label in true_labels]
predicted_labels = [label_mapping[label] for label in predicted_labels]

f1 = f1_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)

In [ ]:

# Print the metrics
metrics = {
    "F1 Score": f1,
    "Precision": precision,
    "Recall": recall,
    "Accuracy": accuracy
}

pprint.pprint(metrics)
print("\n")

{'Accuracy': 0.5462184873949579,
 'F1 Score': 0.6746987951807228,
 'Precision': 0.509090909090909,
 'Recall': 1.0}




In [ ]:
#1850 exemples